In [1]:
# auto reload on file change
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
from PIL import Image
import torchvision.models as models
from tqdm import tqdm

In [3]:
from src.yolov8 import *
from src.yoloTrain import *
from src.yoloDataset import *
from src.yoloLoss import *
from src.utils import *

In [4]:
from src.datasetSetup import srediDataset
srediDataset("datasetoviraw", "datasetyolo.zip")

File downloaded or caches
File unziped
Dataset je uspesno organizovan
Finished dataset customization


In [5]:
features_yolo = nn.Sequential(
    nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2) 
)
def collate_fn(batch):
    images, targets = zip(*batch)
    images = list(images)
    target_tensors = []
    
    for target in targets:
        xx = target[0][:,:,0,:]
        boxes = torch.tensor(xx, dtype=torch.float32)
        labels = torch.tensor(target[1], dtype=torch.int64)
        target_tensor = torch.cat((boxes, labels), dim=2)
        target_tensors.append(target_tensor)
    
    target_tensors = torch.stack(target_tensors)   

    return images, target_tensors

In [6]:
train_img_dir = 'datasetoviraw/datasetyolo.zip_unzip/train/images'
train_labels_dir = 'datasetoviraw/datasetyolo.zip_unzip/train/labels'


train_dataset = YoloV8Dataset(train_img_dir, train_labels_dir, transform=transform_yolo)
train_loader = DataLoader(train_dataset, batch_size=batch_size_yolo, shuffle=True, num_workers=0, collate_fn=collate_fn)

In [7]:
model = YOLOv8(features=features_yolo)
optimizer = optim.Adam(model.fc_layers.parameters(), lr=0.001)

c:\Users\lanal\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lanal\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)
print(device)

cpu


In [9]:
feature_size_yolo = 7  
num_bboxes_yolo = 1 
num_classes_yolo = 4 
lambda_coord = 5.0
lambda_noobj = 0.5
learning_rate = 1e-3

In [10]:
backbone = models.resnet18(pretrained=True)
backbone = nn.Sequential(*list(backbone.children())[:-2]) 

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
yolo_loss = YOLOv8Loss(feature_size=7, num_bboxes=1, num_classes=4, lambda_box=1.0, lambda_cls=1.0, lambda_df=1.0, phi=0.0005) 

In [12]:
criterion = YOLOv8Loss(feature_size=7, num_bboxes=1, num_classes=4, lambda_box=1.0, lambda_cls=1.0, lambda_df=1.0, phi=0.0005)

for epoch in range(num_epochs_yolo):
    epoch_loss = train(model, optimizer, train_loader, device, criterion)
    print(f'Epoch {epoch+1}/{num_epochs_yolo}, Loss: {epoch_loss:.4f}')

Training Epoch:   0%|          | 0/77 [00:00<?, ?it/s]C:\Users\lanal\AppData\Local\Temp\ipykernel_2876\1619369655.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes = torch.tensor(xx, dtype=torch.float32)
C:\Users\lanal\AppData\Local\Temp\ipykernel_2876\1619369655.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(target[1], dtype=torch.int64)
Training Epoch: 100%|██████████| 77/77 [01:08<00:00,  1.13it/s]


Epoch 1/10, Loss: 85.7276


Training Epoch: 100%|██████████| 77/77 [01:07<00:00,  1.15it/s]


Epoch 2/10, Loss: 77.0545


Training Epoch: 100%|██████████| 77/77 [01:05<00:00,  1.17it/s]


...

Epoch 8/10, Loss: 39.5020


Training Epoch: 100%|██████████| 77/77 [01:08<00:00,  1.12it/s]


Epoch 9/10, Loss: 34.8988


Training Epoch: 100%|██████████| 77/77 [01:10<00:00,  1.10it/s]

Epoch 10/10, Loss: 29.4675


In [23]:
torch.save(model.state_dict(), 'yolov8.pth')

In [30]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches

def load_image_and_labels(image_path, label_path):
    try:
        image = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None, None
    
    labels = []
    try:
        with open(label_path, 'r') as f:
            for line in f.readlines():
                labels.append([float(x) for x in line.strip().split()])
    except Exception as e:
        print(f"Error loading labels from {label_path}: {e}")
    
    return image, labels

def plot_image_with_bboxes(image, labels, class_names):
    if image is None or labels is None:
        print("No image or labels to display.")
        return
    
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)
    
    img_width, img_height = image.size
    for label in labels:
        if len(label) != 5:
            print(f"Invalid label format: {label}")
            continue
        
        class_id, x_center, y_center, width, height = label
        x_center *= img_width
        y_center *= img_height
        width *= img_width
        height *= img_height
        x_min = x_center - width / 2
        y_min = y_center - height / 2
        rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x_min, y_min, class_names[int(class_id)], color='white', verticalalignment='top', bbox={'color': 'red', 'pad': 0})
    
    plt.axis('off')
    plt.show()

# Class names for the dataset (modify as needed)
class_names = ["player", "referee", "goalkeeper", "ball"]

# Example usage:
image_dir = "datasetoviraw/datasetyolo.zip_unzip/wtest/images"  # Update the path
label_dir = "datasetoviraw/datasetyolo.zip_unzip/wtest/labels"  # Update the path

# List all images
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

# Check if image_files is empty
if not image_files:
    print(f"No images found in directory: {image_dir}")

# Plot a few example images
for image_file in image_files[:5]:  # Change the number of images to plot
    image_path = os.path.join(image_dir, image_file)
    label_path = os.path.join(label_dir, image_file.replace('.jpg', '.txt'))
    
    image, labels = load_image_and_labels(image_path, label_path)
    if image and labels:
        plot_image_with_bboxes(image, labels, class_names)
    else:
        print(f"Skipping {image_file} due to loading issues.")


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/dataset_yolo8/images/val'